# NS6002 Midterm Programming Questions

There are two questions. Please fill in the correct python programming code to complete the tasks in the questions.

There are code blocks with the following patterns. The easiest way is to input the correct code between two #-bars. **You should find the demonstrations in lab assignments helpful**.
```
########## You Code Here ##########
# Hint: Add a variable to store the refractory period


###################################
```

## Question 1: Modeling a new receptor (15 Points)

Suppose there is a new receptor. It can be modelled by

$$
\frac{dS}{dt} = -\frac{S}{\tau_S} + \alpha \frac{1-S}{\exp\left[-(V - E_L) / (5 \text{mV})\right]}\sum_{t^\text{sp}}\delta \left(t-t^\text{sp}\right).
$$

Here $S$ is the opening of the ion channel, $\alpha$ is the opening parameter for the ion channel, $\tau_S$ is the closing time constant of the ion channels and $t^\text{sp}$s are spiking times. In this exercise, we set $g_S=2.0$ nS, $\alpha = 0.2$, $tau_S = 200$ ms and  $E_S = -83$ mV.

Then, the LIF model becomes

$$
C \frac{dV}{dt} = -g_L \left(V - E_L\right) -g_S S \left(V - E_S\right)+ I_\text{input}.
$$

The following implementation of a LIF neuron is using the Euler's method. Please change the following programming code to incorporate the additional differential equation for $S$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# LIF neuron model parameters
C = 100          # Capacitance (in pF)
g_L = 6.5        # Leakage conductance (in nS)
V_resting = -65  # Resting membrane potential (in mV)
E_L = -65
V_thre = -55     # Threshold potential (in mV)

I_input = 70     # Input current (in pA)

########## You Code Here ##########
# Hint: Additional Parameters
# for the dynamical variable S

g_S = 2.0  # Maximum conductance (in nS)
alpha = 0.2  # Opening parameter
tau_S = 200  # Closing time constant (in ms)
E_S = -83  # Reversal potential (in mV)


###################################



########## You Code Here ##########
# Hint: rewrite this block to
# incorporate the additional
# differerntial equation

def f(t, V, S):
    """
    Differential equation governing the membrane potential of the adaptative LIF neuron model.

    Args:
        t (float): Time
        V (float): Membrane potential
        S (float): Opening of the ion channel

    Returns:
        float: Rate of change of the membrane potential
    """
    return (-g_L * (V - E_L) - g_S * S * (V - E_S) + I_input) / C


###################################

V_0 = -65         # Initial condition
delta_t = 0.1     # Time step
ts = np.arange(0, 1000, delta_t)  # Array of time stamps
V_record = []     # Record of all membrane potentials
V = V_0    # Setting the initial value

########## You Code Here ##########
# Hint: Declare and initialize
# the dynamical varibale S
# Also, we need to have a recording
# list like V_record

S_0 = 0          # Initial synaptic opening
S_record = []    # List to record synaptic opening
S = S_0          # Initial synaptic opening
tspikes = []      # List to store spike times
###################################

for t in ts:

    V_record.append(V)

    ########## You Code Here ##########
    # Hint: Taking the record of S
    S_record.append(S)

    ###################################

    ########## You Code Here ##########
    # Hint: Modify the following code
    # to incorporate the variable S

    V = V + delta_t * f(t, V, S)

    ###################################

    if V > V_thre:    # If the potential is larger than V_thre, we record a spike
        V = V_resting # and reset the potential to resting potential

        ###################################
        # Hint: Updating the variable S
        # becaused of the spike

        tspikes.append(t)  # Record the spike time
        S = S + alpha * (1 - S) * np.exp(-(V - E_L) / 5)  # Update S due to the spike
        ###################################

# Calling the matplotlib for plotting
import matplotlib.pyplot as plt

plt.plot(ts, V_record) # Plotting the graph
plt.xlabel("$t$ / $ms$") # Setting the label for the horizontial axis
plt.ylabel("$V$ / $mV$") # Setting the label for the vertical axis

## Question 2: Postsynaptic Receptors with a Desensitized State  (15 Points)

In the lecture, we have discussed the post-syantpic receptor model with a rising time and a decay time. To respond to the release of neurotransmitters, receptors on the post-synaptic side will open for a short-period of time. To model the opening of receptors, we use the dynamical variable $S$

$$
\frac{dS}{dt} = - \frac{S}{\tau_\text{decay}}+\alpha_S (1-S) X
$$
$$
\tau_\text{rise}\frac{dX}{dt} = - X+\alpha_X \sum_{t^\text{sp}}\delta (t-t^\text{sp})
$$

Here, $\tau_\text{decay}$ is the closing time scale, $\tau_\text{rise}$ is the opening time scale, and $\alpha_X$ and $\alpha_S$ are the tendency to open the ion channel of the receptor.

Suppose we are going to model a receptor with a desensitized state after closing. To model this, we modified the equations to be

$$
\frac{dS}{dt} = - \frac{S}{\tau_\text{decay}}+\alpha_S (1-S-D) X
$$
$$
\tau_\text{rise}\frac{dX}{dt} = - X+\alpha_X \sum_{t^\text{sp}}\delta (t-t^\text{sp})
$$
$$
\frac{dD}{dt} = \frac{S}{\tau_\text{decay}} - \frac{D}{\tau_\text{desen}}
$$

Here $D$ is the portion of receptors of this type in the desensitized state. $\tau_\text{desen} = 100$ ms is the time constant for the receptors to exit the desensitized state. Please change the following programming code to incorporate the additional differential equation for $D$.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters of the receptor
tau_decay = 20
tau_rise = 2
alpha_S = 0.5
alpha_X = 0.5

########## You Code Here ##########
# Hint: Additional Parameters
# for the dynamical variable D
tau_desensitize = 100

###################################



########## You Code Here ##########
# Hint: rewrite this block to
# incorporate the additional
# differerntial equation

# Function to calculate derivatives
def all_dXdt(t, S, X, D):
    dSdt = - S / tau_decay + alpha_S * (1 - S - D) * X
    dXdt = - X / tau_rise
    dDdt = S / tau_decay - D / tau_desensitize  # New equation for D
    return dSdt, dXdt, dDdt

###################################


t_sp = [10, 20, 30, 150, 170, 180]  # the spiking time

S = 0 # initial conditino of S
X = 0 # initial conditino of X
S_record = []
X_record = []

########## You Code Here ##########
# Hint: Declare and initialize
# the dynamical varibale D
# Also, we need to have a recording
# list like V_record
D = 0
D_record = []


###################################


delta_t = 0.2

t_span = np.arange(0, 300.1, delta_t)

# Simulation loop
for t in t_span:
    S_record.append(S)
    X_record.append(X)

    ########## You Code Here ##########
    # Hint: Taking the record of D
    D_record.append(D)


    ###################################



    ########## You Code Here ##########
    # Hint: Modify the following code
    # to incorporate the variable D

    # Calculate derivatives
    dSdt, dXdt, dDdt = all_dXdt(t, S, X, D)

    # Update variables using Euler's method
    S_new = S + delta_t * dSdt
    X_new = X + delta_t * dXdt
    D_new = D + delta_t * dDdt

    ###################################


    # Apply the effect due to spike at t_sp
    if t in t_sp:
        X_new = X_new + alpha_X / tau_rise

    # Update variables
    S = S_new
    X = X_new

    ########## You Code Here ##########
    # Hint: Modify the following code
    # to incorporate the variable D
    D = D_new
    ###################################

# Plotting
plt.plot(t_span, S_record)
plt.xlabel("$t$")
plt.ylabel("$S(t)$")
plt.show()